In [34]:
import numpy as np
import pandas as pd
from tqdm import tqdm

path_to_file = r'D:\REPO\github\ML-Notes\Competition\天池-NLP-新闻文本分类\data\test_a.csv'
df = pd.read_csv(path_to_file)
df = df.head(1000)

with open('./text.txt', 'w') as f:
    for setence in df.text.values:
        f.write(setence + '\n')

## 1. 快速生成tokenizer

In [35]:
# Train a tokenizer
import tokenizers
from transformers import BertTokenizer, LineByLineTextDataset

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

dataset= LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = 'text.txt',
    block_size = 64  # maximum sequence length
)

print('No. of lines: ', len(dataset)) # No of lines in your datset

loading file https://huggingface.co/bert-base-chinese/resolve/main/vocab.txt from cache at C:\Users\Jejune/.cache\huggingface\transformers\36acdf4f3edf0a14ffb2b2c68ba47e93abd9448825202377ddb16dae8114fe07.accd894ff58c6ff7bd4f3072890776c14f4ea34fcc08e79cd88c2d157756dceb
loading file https://huggingface.co/bert-base-chinese/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-chinese/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-chinese/resolve/main/tokenizer_config.json from cache at C:\Users\Jejune/.cache\huggingface\transformers\2dc6085404c55008ba7fc09ab7483ef3f0a4ca2496ccee0cdbf51c2b5d529dff.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading configuration file https://huggingface.co/bert-base-chinese/resolve/main/config.json from cache at C:\Users\Jejune/.cache\huggingface\transformers\6cc404ca8136bc87bae0fb24f2259904943d776a6c5ddc26598bbdc319476f42.0f9bcd8314d841c0

No. of lines:  1000


In [36]:
sentence = "今天天气怎么样"
tokens = ['[CLS]'] + tokenizer.tokenize(sentence) + ['[SEP]']
print(tokens)

['[CLS]', '今', '天', '天', '气', '怎', '么', '样', '[SEP]']


## 2.定义 Bert-base
12 12 768
- 预测掩码词的数量 选择15%要预测的随机词元
- DataCollatorForLanguageModeling API即可方便得以自己的语料定义生成器

In [37]:
from transformers import BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling

config = BertConfig(
    vocab_size=5000,
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    max_position_embeddings=512,
)
model = BertForMaskedLM(config)
print('参数数量:', model.num_parameters())


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

参数数量: 89887880


## 3.定义训练参数

In [38]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./',
    overwrite_output_dir=True,
    num_train_epochs=30,
    per_device_train_batch_size=64,
    save_steps=10000,
    # save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)
trainer.train()
trainer.save_model('./')

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.